In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import datetime
import os
from feature_engineering.tensor_features import develop_features, floating_conv
from sklearn.preprocessing import StandardScaler
from data_handler import LocalToLargeDataLoader
from path_finder import path_sorter

2024-10-24 20:11:34.740529: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 20:11:34.749112: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 20:11:34.817040: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 20:11:34.870022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 20:11:34.935963: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [6]:
data_loader = LocalToLargeDataLoader(print_progress=True)
parsed_data = data_loader.load_raw_data(path="../../Project materials(1)")

Retrieving training data...


In [7]:
print(parsed_data.keys())

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude',
       'longitude', 'vesselId', 'portId', 'etaParsed'],
      dtype='object')


In [11]:
path_dict = path_sorter(parsed_data)

[===================-] 100.0% complete
[====================] 100.0% complete
Concatting dataframes
Number of dataframes: 152532


In [13]:
print(list(path_dict.keys()))

print(path_dict['navstat_0'])
print(path_dict[list(path_dict.keys())[0]][0])


['vesselId', 'portId', 'etaParsed', 'time_0', 'cog_0', 'sog_0', 'latitude_0', 'longitude_0', 'navstat_0', 'rot_0', 'heading_0', 'time_1', 'cog_1', 'sog_1', 'latitude_1', 'longitude_1', 'navstat_1', 'rot_1', 'heading_1', 'time_2', 'cog_2', 'sog_2', 'latitude_2', 'longitude_2', 'navstat_2', 'rot_2', 'heading_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3', 'longitude_3', 'navstat_3', 'rot_3', 'heading_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4', 'navstat_4', 'rot_4', 'heading_4', 'time_5', 'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'navstat_5', 'rot_5', 'heading_5', 'time_6', 'cog_6', 'sog_6', 'latitude_6', 'longitude_6', 'navstat_6', 'rot_6', 'heading_6', 'time_7', 'cog_7', 'sog_7', 'latitude_7', 'longitude_7', 'navstat_7', 'rot_7', 'heading_7', 'time_8', 'cog_8', 'sog_8', 'latitude_8', 'longitude_8', 'navstat_8', 'rot_8', 'heading_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9', 'longitude_9', 'navstat_9', 'rot_9', 'heading_9']
0         1
1         0
2         0
3       